In [51]:
import pandas as pd
import spacy
import gensim
import en_core_web_sm
from bs4 import BeautifulSoup
import unidecode
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,recall_score,f1_score

In [52]:
df=pd.read_csv("blogtext.csv",nrows=2000)

In [53]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [54]:
len(df)

2000

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
id        2000 non-null int64
gender    2000 non-null object
age       2000 non-null int64
topic     2000 non-null object
sign      2000 non-null object
date      2000 non-null object
text      2000 non-null object
dtypes: int64(2), object(5)
memory usage: 109.5+ KB


In [56]:
df["age"]=df["age"].astype(str)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
id        2000 non-null int64
gender    2000 non-null object
age       2000 non-null object
topic     2000 non-null object
sign      2000 non-null object
date      2000 non-null object
text      2000 non-null object
dtypes: int64(1), object(6)
memory usage: 109.5+ KB


In [58]:
df["topic"].value_counts()

Technology              607
indUnk                  452
Student                 403
Engineering             119
Education               118
Sports-Recreation        75
InvestmentBanking        70
Non-Profit               46
Science                  33
BusinessServices         21
Internet                 20
Banking                  16
Communications-Media     14
Arts                      2
Museums-Libraries         2
Accounting                2
Name: topic, dtype: int64

In [59]:
df["gender"].value_counts()

male      1272
female     728
Name: gender, dtype: int64

In [60]:
df["age"].value_counts()

35    607
24    334
15    299
17    147
25    110
33     94
23     93
27     86
14     74
26     43
39     32
16     25
37     19
45     14
41     14
34      6
44      3
Name: age, dtype: int64

In [61]:
df["sign"].value_counts()

Aries          699
Libra          313
Aquarius       286
Scorpio        243
Sagittarius    113
Capricorn       77
Cancer          76
Taurus          76
Leo             55
Virgo           39
Gemini          21
Pisces           2
Name: sign, dtype: int64

In [62]:
df["text"]

0                  Info has been found (+/- 100 pages,...
1                  These are the team members:   Drewe...
2                  In het kader van kernfusie op aarde...
3                        testing!!!  testing!!!          
4                    Thanks to Yahoo!'s Toolbar I can ...
5                    I had an interesting conversation...
6                    Somehow Coca-Cola has a way of su...
7                    If anything, Korea is a country o...
8                    Take a read of this news article ...
9                    I surf the English news sites a l...
10                   Ah, the Korean language...it look...
11                   If you click on my profile you'll...
12                   Last night was pretty fun...mostl...
13                   There is so much that is differen...
14                    urlLink    Here it is, the super...
15                   One thing I love about Seoul (and...
16                    urlLink    Wonderful oh-gyup-sal...
17            

In [63]:
nlp=en_core_web_sm.load()

In [64]:
def preprocessing(text):
    
    #Removing HTML Tags from Blog Data
    soup=BeautifulSoup(text,"html.parser") 
    #BeautifulSoup gives BeautifulSoup object which represent text as nested data structure
    stripped_text=soup.get_text(separator=" ") #Extracts all text from "text"

    #Convert Accented Characters
    unicoded_text=unidecode.unidecode(stripped_text)
    
    #Converting the text to lower case
    text_lowerCase=unicoded_text.lower()
    
    #Replacing more than one space with one space
    text_lowerCase.replace(r"\s\s+",' ')
    
    #Replacing Special characters
    text_replaced=re.sub('[^a-zA-Z.\d\s]', '', text_lowerCase)
    
    #Removing numbers, stopwords and punctuations and creating tokens
    text_tokens=[]
    doc=nlp(text_replaced)
    for token in doc:
        if token.text!="n" and not token.is_stop and not token.is_punct and not token.like_num and not token.is_space and token.is_alpha :
            text_tokens.append(token.lemma_)
    
    
    return(" ".join(text_tokens))
    

In [65]:
texts=[]

In [66]:
for ind in df.index:
    texts.append(preprocessing(df["text"][ind]))

In [67]:
len(texts)

2000

In [68]:
texts[0]

'info find page mb file wait untill team leader process learn html'

In [69]:
df["processed_text"]=texts

In [70]:
df["processed_text"]

0       info find page mb file wait untill team leader...
1       team member drewes van der laag urllink mail r...
2       het kader van kernfusie op aarde maak je eigen...
3                                            testing test
4       thank yahoos toolbar capture url popup mean co...
5       interesting conversation dad morning talk kore...
6       cocacola way sum thing early flagship jingle w...
7       korea country extreme fadbased think come kore...
8       read news article urllink joongang ilbo north ...
9       surf english news site lot look tidbit korea f...
10      ah korean language look difficult figure read ...
11      click profile will notsostartle discovery bear...
12      night pretty fun company keep recently meet co...
13      different have see not travel canada philippin...
14      urllink superfantastic phonebox today great da...
15      thing love seoul mean korea general happen lit...
16      urllink wonderful ohgyupsal favorite pork rest...
17      late k

In [71]:
df.head()

,id,gender,age,topic,sign,date,text,processed_text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info find page mb file wait untill team leader...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team member drewes van der laag urllink mail r...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing test
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thank yahoos toolbar capture url popup mean co...


In [72]:
df["labels"] = df.apply(lambda x: list([x["gender"],
                                        x["age"],
                                        x["topic"],
                                        x["sign"]]),axis=1)     

In [73]:
df.head()

,id,gender,age,topic,sign,date,text,processed_text,labels
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info find page mb file wait untill team leader...,"[male, 15, Student, Leo]"
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team member drewes van der laag urllink mail r...,"[male, 15, Student, Leo]"
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing test,"[male, 15, Student, Leo]"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thank yahoos toolbar capture url popup mean co...,"[male, 33, InvestmentBanking, Aquarius]"


In [74]:
final_df=df.drop(["id","gender","age","topic","sign","date","text"],axis=1)

In [75]:
final_df.head()

,processed_text,labels
0,info find page mb file wait untill team leader...,"[male, 15, Student, Leo]"
1,team member drewes van der laag urllink mail r...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing test,"[male, 15, Student, Leo]"
4,thank yahoos toolbar capture url popup mean co...,"[male, 33, InvestmentBanking, Aquarius]"


In [76]:
freq={}

In [77]:
for ind in final_df["labels"].index:
    for item in final_df["labels"][ind]:
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1           
       

In [78]:
freq

{'male': 1272,
 '15': 299,
 'Student': 403,
 'Leo': 55,
 '33': 94,
 'InvestmentBanking': 70,
 'Aquarius': 286,
 'female': 728,
 '14': 74,
 'indUnk': 452,
 'Aries': 699,
 '25': 110,
 'Capricorn': 77,
 '17': 147,
 'Gemini': 21,
 '23': 93,
 'Non-Profit': 46,
 'Cancer': 76,
 'Banking': 16,
 '37': 19,
 'Sagittarius': 113,
 '26': 43,
 '24': 334,
 'Scorpio': 243,
 '27': 86,
 'Education': 118,
 '45': 14,
 'Engineering': 119,
 'Libra': 313,
 'Science': 33,
 '34': 6,
 '41': 14,
 'Communications-Media': 14,
 'BusinessServices': 21,
 'Sports-Recreation': 75,
 'Virgo': 39,
 'Taurus': 76,
 'Arts': 2,
 'Pisces': 2,
 '44': 3,
 '16': 25,
 'Internet': 20,
 'Museums-Libraries': 2,
 'Accounting': 2,
 '39': 32,
 '35': 607,
 'Technology': 607}

In [79]:
features=final_df["processed_text"]
target=final_df["labels"]

In [80]:
x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.20)

In [81]:
x_train

500     monday start possibility process buy place mee...
149                              yeah have think bit type
428     blogge coz charles say faithful audience settl...
595     here late cookie talk like need decoder ring u...
986     yep s right be invite yahoo tuesday presentati...
1206    go spiderman night like not fun learn go imple...
45      urllink korea times business section editorial...
599     epiphany remmberance epiphany past like parent...
1772    test run potential gig spend past half hour tr...
130     today bible study school boring long day bore ...
77      like s funny common cute sick mind like friend...
1074    new conclusion interesting yeah today early di...
1344    have work saturday horrible esp sunny mean not...
661     be dream day past past have think lot win powe...
847     word word number doom wow yeah s probably good...
471     lull suddenly fill depressive haze assault per...
954     urllink dennis frentsos erika shay engage coup...
748     people

In [82]:
x_test

1091    sigh clue go happen know think perfect situati...
185     s truly like beeeeeee cause oscar mayer wiener...
1672                           johnathan bizarre creature
8       read news article urllink joongang ilbo north ...
72      think be go claim yeoyidoyeouido new hometown ...
1720    hi vacation family topsail beach nc read paper...
906     urllink link new blog link sidebar basically t...
1661    time work right johnathan insane look friend h...
470     embark hour time ill way flinder mornington pe...
1934     excellent johnathan lose mind plan proceed level
447     news small little island gain international at...
1327    probly go swim suit shopping tomorrow hate cuz...
380     result yippee not badly essay better expect co...
17      late korean rumor mill way coquitlam wifes ear...
1755                           angie share result urllink
370     amazing adventure check urllink man soooooo am...
1259    urllink logro entender porque pero encanta url...
1590          

In [83]:
y_train

500            [female, 27, Education, Aquarius]
149               [male, 25, Non-Profit, Cancer]
428                [female, 24, indUnk, Scorpio]
595               [male, 24, Engineering, Libra]
986                  [female, 23, indUnk, Libra]
1206                 [male, 25, Internet, Aries]
45       [male, 33, InvestmentBanking, Aquarius]
599               [male, 24, Engineering, Libra]
1772               [male, 35, Technology, Aries]
130                 [female, 17, Student, Aries]
77                   [female, 14, indUnk, Aries]
1074                [female, 15, Student, Libra]
1344               [female, 25, Student, Taurus]
661               [male, 24, Engineering, Libra]
847             [male, 17, Student, Sagittarius]
471                [female, 24, indUnk, Scorpio]
954                 [male, 14, Student, Scorpio]
748         [male, 24, BusinessServices, Cancer]
973                  [female, 23, indUnk, Libra]
138               [male, 25, Non-Profit, Cancer]
1493               [

In [84]:
y_test

1091                [female, 15, Student, Libra]
185              [female, 33, Banking, Aquarius]
1672               [male, 35, Technology, Aries]
8        [male, 33, InvestmentBanking, Aquarius]
72       [male, 33, InvestmentBanking, Aquarius]
1720               [male, 35, Technology, Aries]
906     [male, 17, Sports-Recreation, Capricorn]
1661               [male, 35, Technology, Aries]
470                [female, 24, indUnk, Scorpio]
1934               [male, 35, Technology, Aries]
447                [female, 24, indUnk, Scorpio]
1327                 [female, 15, indUnk, Aries]
380                [female, 24, indUnk, Scorpio]
17       [male, 33, InvestmentBanking, Aquarius]
1755               [male, 35, Technology, Aries]
370                [female, 24, indUnk, Scorpio]
1259             [female, 15, Student, Aquarius]
1590               [male, 35, Technology, Aries]
18       [male, 33, InvestmentBanking, Aquarius]
1286                [male, 39, Education, Virgo]
1593               [

In [85]:
vectorizer = CountVectorizer(ngram_range=(1,2))

In [86]:
x_train_vectorized=vectorizer.fit_transform(x_train)

In [87]:
x_train_vectorized

<1600x134097 sparse matrix of type '<class 'numpy.int64'>'
	with 240745 stored elements in Compressed Sparse Row format>

In [88]:
for vector in x_train_vectorized:
    print(vector)

  (0, 22206)	1
  (0, 21419)	1
  (0, 94637)	1
  (0, 21436)	1
  (0, 94638)	1
  (0, 8603)	1
  (0, 81884)	1
  (0, 123345)	1
  (0, 122910)	1
  (0, 133208)	1
  (0, 122914)	1
  (0, 36568)	1
  (0, 79957)	1
  (0, 86830)	1
  (0, 95892)	1
  (0, 72054)	1
  (0, 89339)	1
  (0, 133140)	1
  (0, 128206)	1
  (0, 38991)	1
  (0, 127630)	1
  (0, 105417)	1
  (0, 81970)	1
  (0, 19226)	1
  (0, 45415)	1
  :	:
  (0, 45533)	1
  (0, 105382)	3
  (0, 73727)	1
  (0, 39835)	1
  (0, 46985)	4
  (0, 132928)	3
  (0, 65771)	1
  (0, 6407)	1
  (0, 104888)	1
  (0, 83772)	2
  (0, 53717)	3
  (0, 55882)	2
  (0, 116737)	1
  (0, 100708)	1
  (0, 21435)	2
  (0, 45967)	1
  (0, 84300)	1
  (0, 94932)	3
  (0, 73334)	1
  (0, 87769)	4
  (0, 14593)	3
  (0, 91810)	1
  (0, 89793)	1
  (0, 109771)	1
  (0, 75525)	1
  (0, 10552)	1
  (0, 117236)	1
  (0, 51323)	1
  (0, 132854)	1
  (0, 122652)	1
  (0, 10436)	1
  (0, 132808)	1
  (0, 50998)	1
  (0, 117184)	1
  (0, 78328)	1
  (0, 1438)	1
  (0, 45877)	1
  (0, 78458)	1
  (0, 94657)	1
  (0, 131597)	1
  

  (0, 45967)	1
  (0, 100021)	1
  (0, 34530)	1
  (0, 10467)	1
  (0, 131089)	1
  (0, 119824)	1
  (0, 131367)	1
  (0, 45530)	1
  (0, 23282)	1
  (0, 73353)	1
  (0, 43368)	1
  (0, 131273)	1
  (0, 83491)	1
  (0, 126838)	1
  (0, 112887)	1
  (0, 75650)	1
  (0, 32356)	1
  (0, 78782)	1
  (0, 2754)	1
  (0, 79724)	1
  (0, 73017)	1
  (0, 112884)	1
  (0, 35272)	1
  (0, 54472)	1
  (0, 100008)	1
  (0, 131322)	2
  (0, 51355)	1
  (0, 34516)	1
  (0, 23244)	1
  (0, 126796)	1
  (0, 32350)	1
  (0, 112881)	2
  (0, 35256)	1
  (0, 54467)	1
  (0, 99991)	2
  (0, 119546)	1
  (0, 75569)	1
  (0, 124081)	1
  (0, 80005)	1
  (0, 43296)	1
  (0, 2727)	1
  (0, 131055)	5
  (0, 10436)	1
  (0, 45399)	1
  (0, 72866)	1
  (0, 78747)	1
  (0, 50998)	1
  (0, 83418)	1
  (0, 79700)	2
  (0, 73334)	1
  (0, 8522)	1
  (0, 26111)	1
  (0, 108959)	1
  (0, 18590)	1
  (0, 119396)	1
  (0, 45138)	1
  (0, 77455)	1
  (0, 22166)	1
  (0, 23578)	1
  (0, 91256)	1
  (0, 49738)	1
  (0, 131879)	1
  (0, 48349)	1
  (0, 20818)	1
  (0, 131845)	1
  (0, 873

  (0, 6407)	1
  (0, 60205)	1
  (0, 104)	1
  (0, 67152)	1
  (0, 99463)	1
  (0, 59567)	1
  (0, 81390)	1
  (0, 100810)	1
  (0, 43963)	1
  (0, 104021)	1
  (0, 56626)	1
  (0, 87867)	1
  (0, 112552)	1
  (0, 44185)	1
  (0, 29697)	1
  (0, 74849)	1
  (0, 1181)	1
  (0, 111954)	1
  (0, 17577)	1
  (0, 30014)	1
  (0, 119338)	1
  (0, 120321)	1
  (0, 44323)	1
  (0, 115480)	1
  (0, 101086)	1
  (0, 51274)	1
  :	:
  (0, 60747)	1
  (0, 121265)	1
  (0, 34648)	1
  (0, 3929)	1
  (0, 91017)	1
  (0, 109682)	1
  (0, 60181)	1
  (0, 127733)	1
  (0, 122474)	1
  (0, 43827)	1
  (0, 72554)	1
  (0, 26043)	1
  (0, 99435)	1
  (0, 30011)	1
  (0, 29678)	1
  (0, 72)	1
  (0, 72843)	1
  (0, 56613)	1
  (0, 54718)	1
  (0, 128563)	1
  (0, 63826)	1
  (0, 127576)	2
  (0, 50998)	1
  (0, 79700)	1
  (0, 87769)	1
  (0, 11037)	1
  (0, 11421)	1
  (0, 11024)	1
  (0, 130902)	1
  (0, 11201)	1
  (0, 130888)	1
  (0, 97586)	1
  (0, 127702)	1
  (0, 48684)	1
  (0, 4035)	1
  (0, 87269)	1
  (0, 69734)	1
  (0, 114313)	1
  (0, 54349)	1
  (0, 3632

  (0, 45967)	1
  (0, 86372)	1
  (0, 94355)	1
  (0, 24381)	1
  (0, 73303)	1
  (0, 61708)	1
  (0, 44869)	1
  (0, 50268)	1
  (0, 23948)	1
  (0, 87802)	1
  (0, 114343)	1
  (0, 62690)	1
  (0, 324)	1
  (0, 103287)	1
  (0, 12615)	1
  (0, 130542)	1
  (0, 85373)	1
  (0, 5970)	1
  (0, 67801)	1
  (0, 58842)	1
  (0, 5275)	1
  (0, 60235)	1
  (0, 103225)	1
  (0, 117653)	1
  (0, 68410)	1
  (0, 104144)	1
  :	:
  (0, 122801)	1
  (0, 94960)	1
  (0, 49079)	1
  (0, 68721)	1
  (0, 112320)	1
  (0, 23919)	1
  (0, 29418)	1
  (0, 130534)	1
  (0, 111096)	1
  (0, 5483)	1
  (0, 5289)	1
  (0, 50212)	1
  (0, 78628)	1
  (0, 94364)	1
  (0, 62152)	4
  (0, 85272)	1
  (0, 20733)	1
  (0, 44598)	1
  (0, 67787)	2
  (0, 71358)	1
  (0, 114171)	1
  (0, 126875)	1
  (0, 79700)	1
  (0, 117184)	1
  (0, 87769)	1
  (0, 26880)	1
  (0, 74748)	1
  (0, 26646)	1
  (0, 71597)	1
  (0, 96957)	1
  (0, 29779)	1
  (0, 5912)	1
  (0, 5294)	1
  (0, 57296)	1
  (0, 57310)	1
  (0, 107112)	1
  (0, 107125)	1
  (0, 125236)	1
  (0, 3775)	1
  (0, 15866)

  (0, 45967)	1
  (0, 80146)	1
  (0, 40641)	1
  (0, 46192)	1
  (0, 45545)	1
  (0, 128830)	1
  (0, 21147)	1
  (0, 36267)	1
  (0, 107778)	1
  (0, 8651)	1
  (0, 80541)	1
  (0, 129122)	1
  (0, 27692)	1
  (0, 71126)	1
  (0, 86480)	1
  (0, 21442)	1
  (0, 73027)	1
  (0, 16821)	1
  (0, 75104)	1
  (0, 21434)	1
  (0, 84699)	1
  (0, 30447)	1
  (0, 104528)	1
  (0, 84735)	1
  (0, 126981)	1
  (0, 123258)	1
  :	:
  (0, 39851)	2
  (0, 67787)	1
  (0, 81658)	1
  (0, 69892)	1
  (0, 72866)	3
  (0, 80537)	2
  (0, 114171)	1
  (0, 68096)	1
  (0, 126875)	3
  (0, 115601)	4
  (0, 37677)	1
  (0, 86403)	2
  (0, 101916)	3
  (0, 50998)	4
  (0, 8199)	3
  (0, 131819)	2
  (0, 66209)	5
  (0, 56515)	2
  (0, 79700)	12
  (0, 117184)	2
  (0, 42629)	3
  (0, 45533)	3
  (0, 116737)	3
  (0, 45967)	3
  (0, 109771)	1
  (0, 40086)	1
  (0, 121873)	1
  (0, 69271)	1
  (0, 29304)	1
  (0, 108042)	1
  (0, 29337)	1
  (0, 28751)	1
  (0, 83709)	1
  (0, 32424)	1
  (0, 44742)	1
  (0, 50209)	1
  (0, 120396)	1
  (0, 22053)	1
  (0, 37659)	1
  (

...

  (0, 117184)	1
  (0, 9067)	1
  (0, 130851)	1
  (0, 49632)	1
  (0, 92753)	1
  (0, 101211)	1
  (0, 39406)	1
  (0, 131867)	1
  (0, 95115)	1
  (0, 2094)	1
  (0, 33198)	1
  (0, 77005)	1
  (0, 2015)	1
  (0, 76778)	1
  (0, 108120)	1
  (0, 24492)	1
  (0, 86850)	1
  (0, 121348)	1
  (0, 23364)	1
  (0, 88743)	1
  (0, 29409)	1
  (0, 17116)	1
  (0, 123176)	1
  (0, 86009)	1
  (0, 48378)	1
  (0, 69826)	1
  :	:
  (0, 51278)	1
  (0, 127876)	3
  (0, 39851)	1
  (0, 48740)	2
  (0, 67787)	1
  (0, 69892)	1
  (0, 121790)	1
  (0, 78747)	2
  (0, 82095)	1
  (0, 78374)	1
  (0, 101916)	1
  (0, 50998)	4
  (0, 8199)	2
  (0, 50545)	1
  (0, 131819)	1
  (0, 66209)	2
  (0, 108073)	1
  (0, 56515)	1
  (0, 46985)	1
  (0, 132928)	2
  (0, 65771)	1
  (0, 6407)	1
  (0, 116737)	2
  (0, 45967)	2
  (0, 14593)	2
  (0, 132199)	1
  (0, 23696)	1
  (0, 24594)	1
  (0, 110871)	1
  (0, 9466)	1
  (0, 108876)	1
  (0, 127717)	1
  (0, 25122)	1
  (0, 54016)	1
  (0, 96671)	1
  (0, 90448)	1
  (0, 72386)	1
  (0, 25120)	1
  (0, 66389)	1
  (0, 1

  (0, 132928)	1
  (0, 129714)	1
  (0, 54366)	1
  (0, 21888)	1
  (0, 51695)	1
  (0, 56934)	1
  (0, 133378)	1
  (0, 18016)	1
  (0, 102501)	1
  (0, 33593)	1
  (0, 29692)	1
  (0, 20827)	1
  (0, 35751)	1
  (0, 84711)	1
  (0, 18780)	1
  (0, 118118)	1
  (0, 99874)	1
  (0, 29722)	1
  (0, 85)	1
  (0, 52766)	1
  (0, 81162)	1
  (0, 133377)	1
  (0, 58710)	1
  (0, 56930)	1
  (0, 81161)	1
  (0, 42334)	1
  :	:
  (0, 35743)	1
  (0, 79702)	1
  (0, 129667)	1
  (0, 8653)	1
  (0, 107889)	1
  (0, 118110)	1
  (0, 51670)	1
  (0, 42321)	1
  (0, 129838)	1
  (0, 129783)	1
  (0, 117924)	1
  (0, 54229)	1
  (0, 29678)	2
  (0, 72)	1
  (0, 49079)	1
  (0, 102487)	1
  (0, 59972)	1
  (0, 84680)	1
  (0, 33554)	1
  (0, 58698)	1
  (0, 20733)	1
  (0, 38781)	1
  (0, 17922)	1
  (0, 8199)	1
  (0, 79700)	1
  (0, 61621)	1
  (0, 26842)	1
  (0, 130744)	1
  (0, 10419)	1
  (0, 73585)	1
  (0, 19951)	1
  (0, 37132)	1
  (0, 39083)	1
  (0, 130370)	1
  (0, 131395)	1
  (0, 93417)	1
  (0, 113760)	1
  (0, 90048)	1
  (0, 50424)	1
  (0, 3907

  (0, 87769)	7
  (0, 16458)	1
  (0, 26712)	1
  (0, 55416)	1
  (0, 93728)	1
  (0, 61556)	1
  (0, 14938)	1
  (0, 55823)	1
  (0, 32748)	1
  (0, 132803)	1
  (0, 49280)	1
  (0, 77802)	1
  (0, 103761)	1
  (0, 131430)	1
  (0, 69292)	1
  (0, 78825)	1
  (0, 47462)	1
  (0, 89371)	1
  (0, 123169)	1
  (0, 45160)	1
  (0, 69295)	1
  (0, 127861)	1
  (0, 55688)	1
  (0, 111050)	1
  (0, 76608)	1
  (0, 59076)	1
  :	:
  (0, 68721)	1
  (0, 85659)	2
  (0, 58285)	1
  (0, 111012)	1
  (0, 43296)	1
  (0, 20899)	1
  (0, 121393)	1
  (0, 17393)	1
  (0, 103094)	1
  (0, 84680)	1
  (0, 90818)	1
  (0, 26646)	2
  (0, 20733)	2
  (0, 132808)	2
  (0, 44598)	5
  (0, 67787)	2
  (0, 81658)	1
  (0, 78747)	1
  (0, 16440)	1
  (0, 66209)	1
  (0, 54874)	2
  (0, 46985)	1
  (0, 53717)	1
  (0, 45967)	2
  (0, 109771)	1
  (0, 98151)	1
  (0, 10836)	1
  (0, 43306)	1
  (0, 54305)	1
  (0, 25753)	1
  (0, 112916)	1
  (0, 132900)	1
  (0, 48792)	1
  (0, 44237)	1
  (0, 102003)	1
  (0, 92562)	1
  (0, 125913)	1
  (0, 21287)	1
  (0, 20051)	1
  (0

  (0, 109771)	2
  (0, 91376)	1
  (0, 91375)	1
  (0, 8587)	1
  (0, 72788)	1
  (0, 62502)	1
  (0, 10118)	1
  (0, 64007)	1
  (0, 81716)	1
  (0, 106107)	1
  (0, 78561)	1
  (0, 42492)	1
  (0, 132210)	1
  (0, 21156)	1
  (0, 116769)	1
  (0, 114766)	1
  (0, 127239)	1
  (0, 127244)	1
  (0, 20011)	1
  (0, 127919)	1
  (0, 127880)	1
  (0, 114524)	1
  (0, 128161)	1
  (0, 60419)	1
  (0, 11174)	1
  (0, 102348)	1
  :	:
  (0, 80492)	1
  (0, 17393)	1
  (0, 56192)	1
  (0, 84680)	1
  (0, 80125)	1
  (0, 62152)	2
  (0, 114468)	2
  (0, 20972)	1
  (0, 20733)	2
  (0, 100161)	2
  (0, 127876)	3
  (0, 38781)	1
  (0, 11024)	1
  (0, 81658)	1
  (0, 78747)	1
  (0, 126875)	4
  (0, 78374)	3
  (0, 16440)	1
  (0, 8199)	2
  (0, 66209)	2
  (0, 79700)	3
  (0, 10019)	1
  (0, 46985)	1
  (0, 116737)	4
  (0, 45967)	1
  (0, 54297)	1
  (0, 125370)	1
  (0, 37277)	1
  (0, 63147)	1
  (0, 24541)	1
  (0, 106765)	1
  (0, 77447)	1
  (0, 64812)	1
  (0, 122160)	1
  (0, 122161)	1
  (0, 12708)	1
  (0, 133839)	1
  (0, 87017)	1
  (0, 1630)	1


  (0, 116737)	1
  (0, 81296)	1
  (0, 6267)	1
  (0, 96448)	1
  (0, 70151)	1
  (0, 133850)	1
  (0, 41361)	1
  (0, 69664)	1
  (0, 5988)	1
  (0, 82103)	1
  (0, 85884)	1
  (0, 71187)	1
  (0, 128061)	1
  (0, 715)	1
  (0, 1731)	1
  (0, 17110)	1
  (0, 1790)	2
  (0, 69297)	1
  (0, 92187)	1
  (0, 133809)	1
  (0, 770)	1
  (0, 133771)	1
  (0, 763)	1
  (0, 115736)	1
  (0, 41359)	1
  (0, 116625)	1
  :	:
  (0, 99079)	1
  (0, 5289)	1
  (0, 26646)	1
  (0, 98768)	1
  (0, 94364)	3
  (0, 6923)	1
  (0, 130343)	1
  (0, 63826)	1
  (0, 20733)	2
  (0, 100161)	1
  (0, 10436)	1
  (0, 21418)	2
  (0, 127876)	1
  (0, 69892)	1
  (0, 1729)	3
  (0, 82095)	3
  (0, 115601)	2
  (0, 8199)	2
  (0, 66209)	1
  (0, 79700)	1
  (0, 117184)	3
  (0, 132928)	1
  (0, 65771)	1
  (0, 104888)	1
  (0, 116737)	2
  (0, 122281)	1
  (0, 32755)	1
  (0, 98396)	1
  (0, 10410)	1
  (0, 39466)	1
  (0, 51116)	1
  (0, 34004)	1
  (0, 70628)	1
  (0, 96881)	1
  (0, 56332)	1
  (0, 60198)	1
  (0, 58705)	1
  (0, 73588)	1
  (0, 7134)	1
  (0, 42327)	1
  (

In [89]:
x_test_vectorized=vectorizer.transform(x_test)

In [90]:
x_test_vectorized

<400x134097 sparse matrix of type '<class 'numpy.int64'>'
	with 30411 stored elements in Compressed Sparse Row format>

In [91]:
y_train

500            [female, 27, Education, Aquarius]
149               [male, 25, Non-Profit, Cancer]
428                [female, 24, indUnk, Scorpio]
595               [male, 24, Engineering, Libra]
986                  [female, 23, indUnk, Libra]
1206                 [male, 25, Internet, Aries]
45       [male, 33, InvestmentBanking, Aquarius]
599               [male, 24, Engineering, Libra]
1772               [male, 35, Technology, Aries]
130                 [female, 17, Student, Aries]
77                   [female, 14, indUnk, Aries]
1074                [female, 15, Student, Libra]
1344               [female, 25, Student, Taurus]
661               [male, 24, Engineering, Libra]
847             [male, 17, Student, Sagittarius]
471                [female, 24, indUnk, Scorpio]
954                 [male, 14, Student, Scorpio]
748         [male, 24, BusinessServices, Cancer]
973                  [female, 23, indUnk, Libra]
138               [male, 25, Non-Profit, Cancer]
1493               [

In [92]:
multilabel=MultiLabelBinarizer()

In [105]:
y_train_transformed=multilabel.transform(y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:935: UserWarning: unknown class(es) ['44', 'Accounting', 'Arts', 'Museums-Libraries', 'Pisces'] will be ignored
  .format(sorted(unknown, key=str)))


In [106]:
y_test_transformed=multilabel.transform(y_test)

In [107]:
y_train_transformed.shape

(1600, 42)

In [108]:
y_test_transformed.shape

(400, 42)

In [109]:
clf=LogisticRegression(solver="lbfgs")

In [110]:
clf=OneVsRestClassifier(clf)

In [111]:
clf.fit(x_train_vectorized,y_train_transformed)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [112]:
y_test_predict=clf.predict(x_test_vectorized)

In [113]:
y_test_predict.shape

(400, 42)

In [114]:
y_test_transformed.shape

(400, 42)

'micro':Calculate metrics globally by counting the total true positives, 
        false negatives and false positives.

'macro':Calculate metrics for each label, and find their unweighted mean. 
        This does not take label imbalance into account.

'weighted':Calculate metrics for each label, and find their average weighted by support 
           (the number of true instances for each label). 
           This alters ‘macro’ to account for label imbalance; 
           it can result in an F-score that is not between precision and recall.

'samples':Calculate metrics for each instance, and find their average 
          (only meaningful for multilabel classification where this differs from accuracy_score)

In [115]:
accuracy_score(y_test_transformed,y_test_predict)

0.36

In [116]:
precision=precision_score(y_test_transformed,y_test_predict,average="samples")
print(precision)

0.7275416666666666


In [117]:
recall=recall_score(y_test_transformed,y_test_predict,average="samples")
print(recall)

0.52875


In [118]:
f1_score=2*((precision*recall)/(precision+recall))
print(f1_score)

0.6124177473383967


In [123]:
multilabel.classes_

array(['14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34',
       '35', '37', '39', '41', '45', 'Aquarius', 'Aries', 'Banking',
       'BusinessServices', 'Cancer', 'Capricorn', 'Communications-Media',
       'Education', 'Engineering', 'Gemini', 'Internet',
       'InvestmentBanking', 'Leo', 'Libra', 'Non-Profit', 'Sagittarius',
       'Science', 'Scorpio', 'Sports-Recreation', 'Student', 'Taurus',
       'Technology', 'Virgo', 'female', 'indUnk', 'male'], dtype=object)

In [140]:
multilabel.inverse_transform(y_test_transformed)

[('15', 'Libra', 'Student', 'female'),
 ('33', 'Aquarius', 'Banking', 'female'),
 ('35', 'Aries', 'Technology', 'male'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('17', 'Capricorn', 'Sports-Recreation', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('24', 'Scorpio', 'female', 'indUnk'),
 ('35', 'Aries', 'Technology', 'male'),
 ('24', 'Scorpio', 'female', 'indUnk'),
 ('15', 'Aries', 'female', 'indUnk'),
 ('24', 'Scorpio', 'female', 'indUnk'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('24', 'Scorpio', 'female', 'indUnk'),
 ('15', 'Aquarius', 'Student', 'female'),
 ('35', 'Aries', 'Technology', 'male'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('39', 'Education', 'Virgo', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('17', 'Capricorn', 'Sports-Recreation', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', '

In [139]:
multilabel.inverse_transform(y_test_predict)

[('15', 'Libra', 'Student', 'female'),
 ('male',),
 ('35', 'Aries', 'Technology', 'male'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('male',),
 ('17', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('24', 'female'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('15', 'Aries', 'Libra', 'Student', 'female'),
 ('Scorpio', 'female', 'indUnk'),
 ('male',),
 ('35', 'Aries', 'Technology', 'male'),
 ('male',),
 ('male',),
 ('35', 'Aries', 'Technology', 'male'),
 ('33', 'Aquarius', 'InvestmentBanking', 'male'),
 ('male',),
 ('35', 'Aries', 'Technology', 'male'),
 ('17', 'Capricorn', 'Sports-Recreation', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('male',),
 ('male',),
 ('male',),
 ('male',),
 ('33', 'Aquarius', 'Education', 'InvestmentBanking', 'male'),
 ('15', 'Libra', 'Student', 'female'),
 ('35', 'Aries', 'Technology', 'male'),
 ('17', 'male'),
 ('